In [2]:
! pip install category_encoders
! pip install xgboost
! pip install lightgbm
! pip install optuna
! pip install matplotlib
! pip install catboost

  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/1f/e2/495811f12b2e90753fff0e42a07adb0370a725de17cc23a579ac9d3ca67c/category_encoders-2.6.2-py2.py3-none-any.whl.metadata
     ---------------------------------------- 14.0/14.0 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 7.1/7.1 MB 10.1 MB/s eta 0:00:00
     --------------------------------------- 34.1/34.1 MB 10.1 MB/s eta 0:00:00
     ---------------------------------------- 9.1/9.1 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 10.0/10.0 MB 5.7 MB/s eta 0:00:00
     ------------------------------------- 233.8/233.8 kB 14.0 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 10.7 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=0.11 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
   -------------

In [3]:
import category_encoders as ce
import optuna
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import pandas as pd
from sklearn import ensemble
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

c:\users\vsevo\pycharmprojects\pythonproject2\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
moscow_df = pd.read_csv('data.csv')

In [5]:
moscow_df

,updated,price,is_new,rooms,area,area_live,area_kitchen,floor_,floors,lat,lng,address,city,closest_station,dist_to_station,time,address_type
0,2021-02-28 23:30:29,16200000,0,2,58.60,31.0,10.4,11,17,55.663111,37.535233,ул. академика челомея 2,moscow,Воронцовская,0.817926,243061860,ул.
1,2021-02-28 23:30:29,15100000,0,3,70.00,41.7,11.0,6,8,55.744936,37.720906,авиамоторная ул. 30,moscow,Авиамоторная,0.867403,249401111,ул.
2,2021-02-28 23:30:31,59500000,0,3,128.40,92.5,20.0,12,57,55.798480,37.520671,чапаевский пер. 3,moscow,Сокол,0.991473,245344012,пер.
3,2021-02-28 23:30:32,7850000,1,1,41.30,20.0,10.0,2,5,55.809216,37.759246,амурская ул. вл3бс1,moscow,Локомотив,1.641747,245369240,ул.
4,2021-02-28 23:30:34,12600000,0,2,54.40,32.2,9.5,8,17,55.597668,37.519018,вильнюсская ул. 15,moscow,Ясенево,1.857019,249071508,ул.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547020,2022-03-03 15:31:51,7881820,1,0,23.08,9.6,5.6,2,14,55.855774,37.621708,грин парк жк,moscow,Отрадное,2.076289,270789749,жк
547021,2022-03-03 15:31:52,8140728,1,0,23.61,11.1,5.0,3,14,55.855774,37.621708,грин парк жк,moscow,Отрадное,2.076289,270789746,жк
547022,2022-03-03 15:31:52,8989574,1,0,26.62,11.8,6.6,7,14,55.855774,37.621708,грин парк жк,moscow,Отрадное,2.076289,270789741,жк
547023,2022-03-03 15:31:55,12757016,1,1,41.84,12.4,20.3,2,14,55.855774,37.621708,грин парк жк,moscow,Отрадное,2.076289,270789731,жк


In [6]:
moscow_df = moscow_df[moscow_df.price>10000]

In [7]:
X, y = moscow_df.drop(columns=['price', 'address', 'updated', 'lat', 'lng', 'city']), moscow_df['price']

In [9]:
X

,is_new,rooms,area,area_live,area_kitchen,floor_,floors,closest_station,dist_to_station,time,address_type
0,0,2,58.60,31.0,10.4,11,17,Воронцовская,0.817926,243061860,ул.
1,0,3,70.00,41.7,11.0,6,8,Авиамоторная,0.867403,249401111,ул.
2,0,3,128.40,92.5,20.0,12,57,Сокол,0.991473,245344012,пер.
3,1,1,41.30,20.0,10.0,2,5,Локомотив,1.641747,245369240,ул.
4,0,2,54.40,32.2,9.5,8,17,Ясенево,1.857019,249071508,ул.
...,...,...,...,...,...,...,...,...,...,...,...
547020,1,0,23.08,9.6,5.6,2,14,Отрадное,2.076289,270789749,жк
547021,1,0,23.61,11.1,5.0,3,14,Отрадное,2.076289,270789746,жк
547022,1,0,26.62,11.8,6.6,7,14,Отрадное,2.076289,270789741,жк
547023,1,1,41.84,12.4,20.3,2,14,Отрадное,2.076289,270789731,жк


In [10]:
encoder = ce.CatBoostEncoder()
X_cb = encoder.fit_transform(X, y)

In [11]:
X_cb

,is_new,rooms,area,area_live,area_kitchen,floor_,floors,closest_station,dist_to_station,time,address_type
0,0,2,58.60,31.0,10.4,11,17,2.112412e+07,0.817926,243061860,2.112412e+07
1,0,3,70.00,41.7,11.0,6,8,2.112412e+07,0.867403,249401111,1.866206e+07
2,0,3,128.40,92.5,20.0,12,57,2.112412e+07,0.991473,245344012,2.112412e+07
3,1,1,41.30,20.0,10.0,2,5,2.112412e+07,1.641747,245369240,1.747471e+07
4,0,2,54.40,32.2,9.5,8,17,2.112412e+07,1.857019,249071508,1.506853e+07
...,...,...,...,...,...,...,...,...,...,...,...
547020,1,0,23.08,9.6,5.6,2,14,1.350396e+07,2.076289,270789749,1.493327e+07
547021,1,0,23.61,11.1,5.0,3,14,1.350243e+07,2.076289,270789746,1.493321e+07
547022,1,0,26.62,11.8,6.6,7,14,1.350098e+07,2.076289,270789741,1.493314e+07
547023,1,1,41.84,12.4,20.3,2,14,1.349975e+07,2.076289,270789731,1.493309e+07


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_cb, y, test_size=0.2, random_state=13)

In [13]:
def rf_objective_reg(trial):
    from sklearn.ensemble import RandomForestRegressor
   
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 24),
        'min_samples_split': trial.suggest_int('min_samples_split', 5, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 60),
    }

    model = RandomForestRegressor(criterion='squared_error', random_state=13, **params)
    model.fit(X_train, y_train)
    pred_labels = model.predict(X_test)
    ev = mean_absolute_percentage_error(y_test, pred_labels)
    return ev

def xgb_objective_reg(trial):
    from xgboost import XGBRegressor

    param = {
        'max_depth': trial.suggest_int('depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.05),
        "lambda": trial.suggest_float('lambda', 0, 1),
        "alpha": trial.suggest_float('alpha', 0, 1),
        "subsample": trial.suggest_float('subsample', 0.5, 1),
        'tree_method': 'gpu_hist',
        'gpu_id': 0
       
    }

    model = XGBRegressor( **param)
    model.fit(X_train, y_train)
    pred_labels = model.predict(X_test)
    ev = mean_absolute_percentage_error(y_test, pred_labels)
    return ev


def lgbm_objective_reg(trial):
    from lightgbm import LGBMRegressor

    param = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'device': 'gpu',
        'max_depth': trial.suggest_int('depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.05)
    }

    model = LGBMRegressor(**param)
    model.fit(X_train, y_train)
    pred_labels = model.predict(X_test)
    ev = mean_absolute_percentage_error(y_test, pred_labels)
    return ev


def cb_objective_reg(trial):
    from catboost import CatBoostRegressor

    param = {
        'iterations': trial.suggest_int("iterations", 100, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.05),
        'depth': trial.suggest_int('max_depth', 2, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0, 1),
        'silent': True,
        'task_type': "GPU",
        'devices': "0:1",
    }

    model = CatBoostRegressor(**param)
    model.fit(X_train, y_train, silent=True, eval_set=(X_test, y_test), early_stopping_rounds=50)
    pred_labels = model.predict(X_test)
    ev = mean_absolute_percentage_error(y_test, pred_labels)
    return ev

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(xgb_objective_reg, n_trials=800)

[I 2023-08-29 14:37:27,450] A new study created in memory with name: no-name-e5c41445-a5d7-4815-9810-c294713297d9
[I 2023-08-29 14:37:29,062] Trial 0 finished with value: 0.1428051028776049 and parameters: {'depth': 6, 'learning_rate': 0.022646208557249286, 'lambda': 0.9136078718014699, 'alpha': 0.5217890900939345, 'subsample': 0.5364345016722165}. Best is trial 0 with value: 0.1428051028776049.
[I 2023-08-29 14:37:30,059] Trial 1 finished with value: 0.26517333631861595 and parameters: {'depth': 4, 'learning_rate': 0.012344240011818117, 'lambda': 0.8728582618910805, 'alpha': 0.2299610553659437, 'subsample': 0.7976493428432188}. Best is trial 0 with value: 0.1428051028776049.
[I 2023-08-29 14:37:30,842] Trial 2 finished with value: 0.3012183865318729 and parameters: {'depth': 2, 'learning_rate': 0.01090897818693135, 'lambda': 0.7696354167811501, 'alpha': 0.6657456809276631, 'subsample': 0.9426106000182579}. Best is trial 0 with value: 0.1428051028776049.
[I 2023-08-29 14:37:32,485] Tri

In [14]:
study = optuna.create_study(direction="minimize")
study.optimize(lgbm_objective_reg, n_trials=800)

[I 2023-08-29 16:51:05,268] A new study created in memory with name: no-name-df856eb8-61e1-445b-b49a-5f4190127cfc


[LightGBM] [Warning] feature_fraction is set=0.620550915538727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.620550915538727
[LightGBM] [Warning] lambda_l1 is set=0.1776731469289237, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1776731469289237
[LightGBM] [Warning] lambda_l2 is set=0.1014644031286385, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1014644031286385
[LightGBM] [Warning] bagging_fraction is set=0.23925531425840824, subsample=1.0 will be ignored. Current value: bagging_fraction=0.23925531425840824
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set=0.620550915538727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.620550915538727
[LightGBM] [Warning] lambda_l1 is set=0.1776731469289237, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1776731469289237
[LightGBM] [Warning] lambda_l2 is set=0

[I 2023-08-29 16:51:27,558] Trial 0 finished with value: 0.5600493199912698 and parameters: {'lambda_l1': 0.1776731469289237, 'lambda_l2': 0.1014644031286385, 'num_leaves': 208, 'feature_fraction': 0.620550915538727, 'bagging_fraction': 0.23925531425840824, 'bagging_freq': 8, 'min_child_samples': 61, 'depth': 2, 'learning_rate': 0.008183169973273409}. Best is trial 0 with value: 0.5600493199912698.


[LightGBM] [Warning] feature_fraction is set=0.2208147007977972, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2208147007977972
[LightGBM] [Warning] lambda_l1 is set=0.2561583468367563, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2561583468367563
[LightGBM] [Warning] lambda_l2 is set=0.38689707246889843, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.38689707246889843
[LightGBM] [Warning] bagging_fraction is set=0.6519906999440717, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6519906999440717
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] feature_fraction is set=0.2208147007977972, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2208147007977972
[LightGBM] [Warning] lambda_l1 is set=0.2561583468367563, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2561583468367563
[LightGBM] [Warning] lambda_l2 is

[I 2023-08-29 16:51:35,424] Trial 1 finished with value: 0.5023248044433551 and parameters: {'lambda_l1': 0.2561583468367563, 'lambda_l2': 0.38689707246889843, 'num_leaves': 234, 'feature_fraction': 0.2208147007977972, 'bagging_fraction': 0.6519906999440717, 'bagging_freq': 14, 'min_child_samples': 1, 'depth': 5, 'learning_rate': 0.009491265323998415}. Best is trial 1 with value: 0.5023248044433551.


...

[LightGBM] [Warning] feature_fraction is set=0.7668822677184215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7668822677184215
[LightGBM] [Warning] lambda_l1 is set=0.21844060002365762, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21844060002365762
[LightGBM] [Warning] lambda_l2 is set=0.24360563643291527, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.24360563643291527
[LightGBM] [Warning] bagging_fraction is set=0.9462386572168595, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9462386572168595
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.7668822677184215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7668822677184215
[LightGBM] [Warning] lambda_l1 is set=0.21844060002365762, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.21844060002365762
[LightGBM] [Warning] lambda_l2 

[I 2023-08-29 17:43:31,974] Trial 148 finished with value: 0.11148727300759287 and parameters: {'lambda_l1': 0.21844060002365762, 'lambda_l2': 0.24360563643291527, 'num_leaves': 503, 'feature_fraction': 0.7668822677184215, 'bagging_fraction': 0.9462386572168595, 'bagging_freq': 1, 'min_child_samples': 7, 'depth': 10, 'learning_rate': 0.045104343594530256}. Best is trial 140 with value: 0.10940168079695908.


[LightGBM] [Warning] feature_fraction is set=0.8016913379892363, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8016913379892363
[LightGBM] [Warning] lambda_l1 is set=0.2366029990551643, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2366029990551643
[LightGBM] [Warning] lambda_l2 is set=0.27974249330191536, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.27974249330191536
[LightGBM] [Warning] bagging_fraction is set=0.9274655355084209, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9274655355084209
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.8016913379892363, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8016913379892363
[LightGBM] [Warning] lambda_l1 is set=0.2366029990551643, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2366029990551643
[LightGBM] [Warning] lambda_l2 is s

[W 2023-08-29 17:43:38,686] Trial 149 failed with parameters: {'lambda_l1': 0.2366029990551643, 'lambda_l2': 0.27974249330191536, 'num_leaves': 512, 'feature_fraction': 0.8016913379892363, 'bagging_fraction': 0.9274655355084209, 'bagging_freq': 0, 'min_child_samples': 13, 'depth': 10, 'learning_rate': 0.026381604809910607} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\users\vsevo\pycharmprojects\pythonproject2\venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\vsevo\AppData\Local\Temp\ipykernel_3084\1644584269.py", line 56, in lgbm_objective_reg
    model.fit(X_train, y_train)
  File "c:\users\vsevo\pycharmprojects\pythonproject2\venv\lib\site-packages\lightgbm\sklearn.py", line 1062, in fit
    init_model=init_model
  File "c:\users\vsevo\pycharmprojects\pythonproject2\venv\lib\site-packages\lightgbm\sklearn.py", line 851, in fit
    callbacks=callbacks
  Fil

KeyboardInterrupt: 

In [15]:
study1 = optuna.create_study(direction="minimize")
study1.optimize(cb_objective_reg, n_trials=50)

[I 2023-08-29 17:43:51,706] A new study created in memory with name: no-name-cc63a406-c685-4507-be84-f87bcd49e8a3
[I 2023-08-29 17:44:17,708] Trial 0 finished with value: 0.14659023189578757 and parameters: {'iterations': 2285, 'learning_rate': 0.029416835583228888, 'max_depth': 2, 'l2_leaf_reg': 0.7827301577410559}. Best is trial 0 with value: 0.14659023189578757.
[I 2023-08-29 17:44:43,042] Trial 1 finished with value: 0.12695658878980526 and parameters: {'iterations': 1149, 'learning_rate': 0.03406152940610912, 'max_depth': 6, 'l2_leaf_reg': 0.9702562626022908}. Best is trial 1 with value: 0.12695658878980526.
[I 2023-08-29 17:46:35,029] Trial 2 finished with value: 0.10193737545174451 and parameters: {'iterations': 2064, 'learning_rate': 0.040131647011355055, 'max_depth': 10, 'l2_leaf_reg': 0.5809000887906433}. Best is trial 2 with value: 0.10193737545174451.
[I 2023-08-29 17:46:42,374] Trial 3 finished with value: 0.3670002968173224 and parameters: {'iterations': 140, 'learning_ra

KeyboardInterrupt: 

In [16]:
lgbm_param = {'lambda_l1': 0.1873614189651116, 'lambda_l2': 0.30002810690969617, 'num_leaves': 462, 'feature_fraction': 0.775150906035067, 'bagging_fraction': 0.9126199831365078, 'bagging_freq': 8, 'min_child_samples': 15, 'depth': 10, 'learning_rate': 0.04885918962472099}

In [17]:
xgb_param = {'depth': 10, 'learning_rate': 0.04997590541942759, 'lambda': 0.0704821218024878, 'alpha': 0.38527237892202787, 'subsample': 0.9919547695879909}

In [18]:
cb_param =  {'iterations': 2974, 'learning_rate': 0.049933384664045635, 'max_depth': 10, 'l2_leaf_reg': 0.4146524901999288}

In [21]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

modellgbm = LGBMRegressor(**lgbm_param).fit(X_train, y_train)
modelxg = XGBRegressor(**xgb_param).fit(X_train, y_train)
modelcb = CatBoostRegressor(**cb_param).fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] feature_fraction is set=0.775150906035067, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.775150906035067
[LightGBM] [Warning] lambda_l1 is set=0.1873614189651116, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1873614189651116
[LightGBM] [Warning] lambda_l2 is set=0.30002810690969617, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.30002810690969617
[LightGBM] [Warning] bagging_fraction is set=0.9126199831365078, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9126199831365078
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] feature_fraction is set=0.775150906035067, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.775150906035067
[LightGBM] [Warning] lambda_l1 is set=0.1873614189651116, reg_alpha=0.0 will be 

In [23]:
mean_absolute_percentage_error (y, (modellgbm.predict(X_cb)+modelxg.predict(X_cb)+modelcb.predict(X_cb))/3)

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] feature_fraction is set=0.775150906035067, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.775150906035067
[LightGBM] [Warning] lambda_l1 is set=0.1873614189651116, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1873614189651116
[LightGBM] [Warning] lambda_l2 is set=0.30002810690969617, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.30002810690969617
[LightGBM] [Warning] bagging_fraction is set=0.9126199831365078, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9126199831365078
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8


0.10426068848332776

In [24]:
import pickle

pickle.dump(modellgbm, open('modellgbm.pkl', 'wb'))
pickle.dump(modelxg, open('modelxg.pkl', 'wb'))
pickle.dump(modelcb, open('modelcb.pkl', 'wb'))

In [26]:
pickle.dump(encoder, open('encoder_cb.pkl', 'wb'))

In [27]:
modelxg.save_model('modelxg.pkl')